In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D , BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from utils import 
import pickle

In [ ]:
train_data , val_data =load_data()

In [ ]:
visualize_samples(train_data)

In [ ]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = True  # unFreeze the base

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

# First dense block
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)

# Second dense block
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

# Output layer
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

checkpoint = ModelCheckpoint("/output/best_model.h5", save_best_only=True, monitor='val_accuracy')

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stop, checkpoint]
)

with open("/output/history.pkl", "wb") as f:
    pickle.dump(history.history, f)

In [ ]:
with open("/output/history.pkl", "rb") as f:
    history_dict = pickle.load(f)

visualize_accuracy_loss(history_dict)